In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 5400)
pd.set_option('display.max_colwidth', 500)

In [2]:
df = pd.read_csv('GSAF5.csv', encoding='cp1252')

In [3]:
#list(df.columns)

### Rename column names

In [4]:
df = df.rename(columns={'Sex ': 'Gender', 'Species ': 'Species', 'Fatal (Y/N)': 'Fatal'})
#list(df.columns)

### Checking for duplicates

In [5]:
df.duplicated().sum()

0

### Checking for missing values

In [6]:
null_cols = df.isnull().sum()
#null_cols[null_cols > 0]

In [7]:
null_perc = null_cols[null_cols > 0] / len(df) 
to_drop = null_perc[null_perc > .9].index
to_drop

Index(['Unnamed: 22', 'Unnamed: 23'], dtype='object')

In [8]:
df = df.drop(to_drop, axis=1)

### Check for columns that have identical values

In [9]:
sum(df['Case Number'] == df['Case Number.1'])/len(df)

0.99783044058745

In [10]:
sum(df['Case Number'] == df['Case Number.2'])/len(df)

0.9996662216288384

In [11]:
sum(df['Case Number.1'] == df['Case Number.2'])/len(df)

0.9981642189586115

In [12]:
# Check how many href values are identical with href formula values
sum(df['href'] == df['href formula'])/len(df)

0.9909879839786382

In [13]:
# Check if pdf column  exists in href column

counter1 = []
for i in range(len(df)):
    if any(df['href'].str.contains(df['pdf'][i])):
        counter1.append(df['pdf'][i])

In [14]:
len(counter1)/len(df)

1.0

In [15]:
# Check if Pdf exists in Case Number column

counter2 = []
for i in range(len(df)):
     if any(df['pdf'].str.contains(df['Case Number'][i])):
        counter2.append(df['Case Number'][i])

In [16]:
len(counter2) / len(df)

0.9771361815754339

In [17]:
# Remove href, href columns because the name of the pdf is already in href and href formula
# Pdf name can provide us with the URL so no need to keep the other two
# Case number contains the date and in some occasions the name of the pdf
# The originak order column is just the indexing reveresed but starting with 2 instead of 1

df = df.drop(columns=['Case Number.1', 'Case Number.2', 'href formula', 'href', 'Case Number', 'original order'])

###  Cleaning of Fatal column

In [18]:
df['Fatal'].value_counts()

N          4315
Y          1552
UNKNOWN      94
 N            8
#VALUE!       1
N             1
F             1
n             1
Name: Fatal, dtype: int64

In [19]:
df.Fatal.replace({'N': 'No', ' N': 'No', 'N ': 'No', 'n': 'No', 'Y': 'Yes'}, inplace=True)
df.Fatal.replace({'UNKNOWN': 'Unknown', '#VALUE!': np.nan, 'F': np.nan }, inplace=True)
df['Fatal'].value_counts()

No         4325
Yes        1552
Unknown      94
Name: Fatal, dtype: int64

###  Cleaning of Gender (Sex) column

In [20]:
df.Gender.value_counts()

M      4835
F       585
M         2
.         1
lli       1
N         1
Name: Gender, dtype: int64

In [21]:
df.Gender.replace({'M': 'Male', 'M ': 'Male', 'F': 'Female', '.': np.nan, 'lli': np.nan, 'N': np.nan}, inplace=True)
df.Gender.value_counts()

Male      4837
Female     585
Name: Gender, dtype: int64

### Cleaning the Type column 

In [22]:
df.Type.value_counts()

Unprovoked      4386
Provoked         557
Invalid          519
Sea Disaster     220
Boat             200
Boating          110
Name: Type, dtype: int64

In [23]:
df.Type.replace('Boating', 'Boat', inplace=True)
df.Type.value_counts()

Unprovoked      4386
Provoked         557
Invalid          519
Boat             310
Sea Disaster     220
Name: Type, dtype: int64

### Cleaning the Name column

In [28]:
#Removing all non relevant values from Name column

df.Name.replace(['male', 'female', 'boy', '2 males', 'boat', 'Anonymous', 'child', 'sailor',
                 'girl', 'a sailor', 'fisherman', 'Unidentified', 'males', 'a native', 'a pearl diver', 
                 'Unknown', 'Arab boy', 'black male', 'a soldier', '2 fishermen', 'Japanese diver',
                 ' male', 'dinghy', 'aboriginal male', 'unkown', 'Zulu male', 'Russian male',
                 '2 women', 'native boy', 'woman', '3 males', 'Indian female', 'unknown', '"a youth"',
                 '11 stoways'], np.nan, inplace=True)

In [29]:
df.Name.value_counts()

M.C.                                    3
Andre Hartman                           3
Fijian girl                             2
Gerjo Van Niekerk                       2
Robert Martin                           2
                                       ..
Sudesh Sarjoo                           1
boat, occupants: Jacob Kruger & crew    1
Conrad Botha                            1
John William Eke                        1
William McLeod                          1
Name: Name, Length: 4975, dtype: int64

In [31]:
GSAF5_clean = df.to_csv(r'C:\Users\GeorgiosKP\projects\Pandas\GSAF5_clean.csv', index=False)
